In [1]:
# Filter tensorflow version warnings
import os
# https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import warnings
# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import logging
tf.get_logger().setLevel(logging.ERROR)

In [2]:
import gym
from stable_baselines.common.policies import CnnPolicy #, MlpPolicy, CnnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv 
from stable_baselines import PPO2

from stable_baselines.common.evaluation import evaluate_policy as test
from stable_baselines.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold


In [3]:
## Choose one agent, see Docu for description
agent='CarRacing-v0'
agent='CarRacing-v1'
agent='CarRacing-v2'

# Stop training when the model reaches the reward threshold
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold = 150, verbose=1)

In [4]:
## This environment param  
## Changing these makes models incompatible!!
game_color = 1
indicators = True
fpst = 4
skip = 3
actions = [[0, 0, 0], [-0.4, 0, 0], [0.4, 0, 0], [0, 0.6, 0], [0, 0, 0.8]]

In [5]:
## This model param
use = 100       # number of times to use same track [1,100]
ept = 15       # different starting points on same track [1,20]
track_complexity = 12
patience = 2.0
seed = 1000

#using follow_centerline for this first leg of training
REWARD = [-0.0, 0.1, 0.0, 0.0, 1.0, 100, -20, -100, -50]

if agent=='CarRacing-v2': 
    env1 = gym.make(agent, #seed=seed, 
        use_track = use,       
        episodes_per_track = ept,  
        tr_complexity = track_complexity, 
        patience = patience,
        game_color=game_color,
        indicators = indicators,
        discre = actions,     #passing custom actions
        frames_per_state = fpst,
        skip_frames = skip,
        f_reward = REWARD )   #passing a custom reward function
else: 
    env1 = gym.make(agent)

env1 = DummyVecEnv([lambda: env1])
env1.metadata

{'render.modes': ['human', 'rgb_array', 'state_pixels'],
 'FPS, 1/timebase': 33.333333333333336,
 'Zoom_level': 1.7,
 'Flight start': False,
 'show_track_1st': False,
 'state_pixels frame size': [96, 96]}

In [6]:
print(env1.action_space)
print(env1.observation_space)

Discrete(5)
Box(0, 255, (96, 96, 4), uint8)


In [7]:
## This training param
batch_size = 256
updates = 1000
epochs = 4

model = PPO2(CnnPolicy, env1, verbose=1, n_steps=batch_size, # seed=314, n_cpu_tf_sess=1,
             gamma=0.99, learning_rate=0.00025, nminibatches=epochs, ent_coef=0.01, vf_coef=0.5) 


In [8]:
## Separate evaluation env
test_freq = 50      #policy updates until evaluation
test_episodes_per_track = 10   #number of starting points on test_track
eval_log = './evals/'

env_test = gym.make(agent, seed=int(3.14*seed), 
        use_track = 1,        #change test track after 1 ept round
        episodes_per_track = test_episodes_per_track,  
        tr_complexity = 12,   # test on a medium complexity track
        patience = patience,  #1.0,
        game_color=game_color,
        indicators = indicators,
        discre = actions,
        frames_per_state = fpst,
        skip_frames = skip   )

env_test = DummyVecEnv([lambda: env_test])

eval_callback = EvalCallback(env_test, callback_on_new_best=callback_on_best,  #None,
                             n_eval_episodes = test_episodes_per_track*2, eval_freq = test_freq*batch_size,
                             best_model_save_path=eval_log, log_path=eval_log, 
                             deterministic=True, render=False)


In [9]:
##Independent test routine
#reward_test, epis = test(model, env_test, n_eval_episodes=test_episodes_per_track, 
#                         deterministic=True, render=False, callback=None, reward_threshold=100, 
#                         return_episode_rewards=True)
#reward_test

In [10]:
## Training #1

model.learn(total_timesteps = updates*batch_size, log_interval=1, callback=eval_callback)


Track generation: 1252..1568 -> 316-tiles track, complex 12
1  cut by time without progress. Steps 157  %advance 1.5  played reward 13.71  last penalty -20
-------------------------------------
| approxkl           | 0.002542828  |
| clipfrac           | 0.0          |
| explained_variance | -0.00626     |
| fps                | 48           |
| n_updates          | 1            |
| policy_entropy     | 1.6068904    |
| policy_loss        | -0.010900296 |
| serial_timesteps   | 256          |
| time_elapsed       | 0            |
| total_timesteps    | 256          |
| value_loss         | 7.487932     |
-------------------------------------
--------------------------------------
| approxkl           | 0.0031470456  |
| clipfrac           | 0.0126953125  |
| explained_variance | -0.149        |
| fps                | 55            |
| n_updates          | 2             |
| policy_entropy     | 1.596546      |
| policy_loss        | -0.0038903551 |
| serial_timesteps   | 512           |

12  cut by time without progress. Steps 229  %advance 1.9  played reward 10.74  last penalty -20
-------------------------------------
| approxkl           | 0.0050288932 |
| clipfrac           | 0.07421875   |
| explained_variance | 0.501        |
| fps                | 54           |
| n_updates          | 16           |
| policy_entropy     | 1.3482565    |
| policy_loss        | 0.0003911123 |
| serial_timesteps   | 4096         |
| time_elapsed       | 72.4         |
| total_timesteps    | 4096         |
| value_loss         | 7.477728     |
-------------------------------------
13  cut by time without progress. Steps 212  %advance 2.5  played reward 10.77  last penalty -20
14  cut by time without progress. Steps 187  %advance 1.5  played reward 10.74  last penalty -20
-------------------------------------
| approxkl           | 0.010636956  |
| clipfrac           | 0.16699219   |
| explained_variance | 0.591        |
| fps                | 54           |
| n_updates          | 17

30  cut by time without progress. Steps 98  %advance 0.9  played reward 9.44  last penalty -20
--------------------------------------
| approxkl           | 0.0018375893  |
| clipfrac           | 0.0087890625  |
| explained_variance | 0.0677        |
| fps                | 53            |
| n_updates          | 29            |
| policy_entropy     | 1.3937393     |
| policy_loss        | -0.0037585413 |
| serial_timesteps   | 7424          |
| time_elapsed       | 133           |
| total_timesteps    | 7424          |
| value_loss         | 20.465916     |
--------------------------------------
31  cut by time without progress. Steps 139  %advance 1.2  played reward 12.22  last penalty -20
--------------------------------------
| approxkl           | 0.0031281535  |
| clipfrac           | 0.0           |
| explained_variance | -0.0061       |
| fps                | 54            |
| n_updates          | 30            |
| policy_entropy     | 1.3275113     |
| policy_loss        | 0.000

42  cut by time without progress. Steps 94  %advance 0.9  played reward 8.17  last penalty -20
--------------------------------------
| approxkl           | 0.0044078277  |
| clipfrac           | 0.07714844    |
| explained_variance | -0.046        |
| fps                | 51            |
| n_updates          | 44            |
| policy_entropy     | 1.4091861     |
| policy_loss        | -0.0036994016 |
| serial_timesteps   | 11264         |
| time_elapsed       | 205           |
| total_timesteps    | 11264         |
| value_loss         | 12.50644      |
--------------------------------------
--------------------------------------
| approxkl           | 0.0067174113  |
| clipfrac           | 0.09375       |
| explained_variance | 0.666         |
| fps                | 53            |
| n_updates          | 45            |
| policy_entropy     | 1.508414      |
| policy_loss        | -0.0072638313 |
| serial_timesteps   | 11520         |
| time_elapsed       | 210           |
| total_

-------------------------------------
| approxkl           | 0.005221431  |
| clipfrac           | 0.02734375   |
| explained_variance | 0.951        |
| fps                | 54           |
| n_updates          | 55           |
| policy_entropy     | 1.5835286    |
| policy_loss        | -0.006784471 |
| serial_timesteps   | 14080        |
| time_elapsed       | 297          |
| total_timesteps    | 14080        |
| value_loss         | 0.1018676    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0038716006 |
| clipfrac           | 0.026367188  |
| explained_variance | 0.536        |
| fps                | 54           |
| n_updates          | 56           |
| policy_entropy     | 1.582422     |
| policy_loss        | -0.003605084 |
| serial_timesteps   | 14336        |
| time_elapsed       | 301          |
| total_timesteps    | 14336        |
| value_loss         | 0.33087024   |
-------------------------------------
------------

...

-------------------------------------
| approxkl           | 0.004328994  |
| clipfrac           | 0.048828125  |
| explained_variance | 0.787        |
| fps                | 45           |
| n_updates          | 935          |
| policy_entropy     | 0.4785681    |
| policy_loss        | -0.010149099 |
| serial_timesteps   | 239360       |
| time_elapsed       | 5.95e+03     |
| total_timesteps    | 239360       |
| value_loss         | 3.7007349    |
-------------------------------------
614  cut by time without progress. Steps 438  %advance 56.8  played reward 70.23  last penalty -20
-------------------------------------
| approxkl           | 0.003957649  |
| clipfrac           | 0.05078125   |
| explained_variance | 0.817        |
| fps                | 49           |
| n_updates          | 936          |
| policy_entropy     | 0.37844515   |
| policy_loss        | -0.009304414 |
| serial_timesteps   | 239616       |
| time_elapsed       | 5.95e+03     |
| total_timesteps    | 2396

624  Finalized in Steps 632  with return=total reward 237.62378608529912
1  Finalized in Steps 484  with return=total reward 150.82779922779915
2  Finalized in Steps 484  with return=total reward 150.82779922779915
3  Finalized in Steps 530  with return=total reward 145.8416988416987
4  Finalized in Steps 499  with return=total reward 149.32779922779918
5  Finalized in Steps 489  with return=total reward 150.32779922779918
6  Finalized in Steps 494  with return=total reward 149.82779922779918
7  Finalized in Steps 482  with return=total reward 151.02779922779916
8  Finalized in Steps 484  with return=total reward 150.82779922779918
9  Finalized in Steps 484  with return=total reward 150.82779922779918
10  Finalized in Steps 505  with return=total reward 148.72779922779915
Track generation: 864..1083 -> 219-tiles track, complex 12
1  cut by time without progress. Steps 326  %advance 61.4  played reward 28.05  last penalty -20
2  cut by time without progress. Steps 295  %advance 53.2  pl

631  cut by time without progress. Steps 320  %advance 40.6  played reward 48.75  last penalty -20
--------------------------------------
| approxkl           | 0.0019795455  |
| clipfrac           | 0.028320312   |
| explained_variance | 0.898         |
| fps                | 53            |
| n_updates          | 962           |
| policy_entropy     | 0.3368936     |
| policy_loss        | -0.0042062607 |
| serial_timesteps   | 246272        |
| time_elapsed       | 6.2e+03       |
| total_timesteps    | 246272        |
| value_loss         | 9.565037      |
--------------------------------------
632  cut by time without progress. Steps 285  %advance 34.6  played reward 40.99  last penalty -20
--------------------------------------
| approxkl           | 0.0007643148  |
| clipfrac           | 0.0078125     |
| explained_variance | 0.977         |
| fps                | 50            |
| n_updates          | 963           |
| policy_entropy     | 0.31488168    |
| policy_loss        |

641  cut by time without progress. Steps 667  %advance 96.8  played reward 134.54  last penalty -20
-------------------------------------
| approxkl           | 0.0023252321 |
| clipfrac           | 0.034179688  |
| explained_variance | -0.0116      |
| fps                | 50           |
| n_updates          | 977          |
| policy_entropy     | 0.16928202   |
| policy_loss        | 0.0020071422 |
| serial_timesteps   | 250112       |
| time_elapsed       | 6.28e+03     |
| total_timesteps    | 250112       |
| value_loss         | 25.849062    |
-------------------------------------
--------------------------------------
| approxkl           | 0.0060650627  |
| clipfrac           | 0.053710938   |
| explained_variance | -0.64         |
| fps                | 50            |
| n_updates          | 978           |
| policy_entropy     | 0.20768847    |
| policy_loss        | -0.0017528993 |
| serial_timesteps   | 250368        |
| time_elapsed       | 6.28e+03      |
| total_timestep

-------------------------------------
| approxkl           | 0.004957715  |
| clipfrac           | 0.0546875    |
| explained_variance | 0.676        |
| fps                | 46           |
| n_updates          | 992          |
| policy_entropy     | 0.5258568    |
| policy_loss        | 4.085258e-05 |
| serial_timesteps   | 253952       |
| time_elapsed       | 6.35e+03     |
| total_timesteps    | 253952       |
| value_loss         | 7.6524053    |
-------------------------------------
-------------------------------------
| approxkl           | 0.010534166  |
| clipfrac           | 0.061523438  |
| explained_variance | -2.32        |
| fps                | 49           |
| n_updates          | 993          |
| policy_entropy     | 0.20946847   |
| policy_loss        | -0.004278073 |
| serial_timesteps   | 254208       |
| time_elapsed       | 6.36e+03     |
| total_timesteps    | 254208       |
| value_loss         | 1.1979743    |
-------------------------------------
650  cut by 

In [11]:
import pickle
root = 'ppo_cnn_gym-mod_'

file = root+'c{:d}_f{:d}_s{:d}_{}_u{:d}_e{:d}_p{}_bs{:d}'.format(game_color,fpst,skip,indicators,use,ept,patience,batch_size)

model.save(file, cloudpickle=True)
param_list=model.get_parameter_list()


In [12]:
#env1.reset_track()
env1.close()

In [13]:
## This model param  #2
use = 5       # number of times to use same track [1,100]
ept = 10       # different starting points on same track [1,20]
#patience = 2.0
seed = 20000
track_complexity = 12
updates = 500

if agent=='CarRacing-v2': 
    env2 = gym.make(agent, seed=seed, 
        use_track = use,       
        episodes_per_track = ept,  
        tr_complexity = track_complexity, 
        patience = patience,
        game_color=game_color,
        indicators = indicators,
        discre = actions,
        frames_per_state = fpst,
        skip_frames = skip   )  #here I use STD_REWARD, so no param needed
else: 
    env2 = gym.make(agent)

env2 = DummyVecEnv([lambda: env2])

In [14]:
## Training  #2
new_mod = False  #to change batch_size you need a new model !!
updates = 500

if new_mod:
    batch_size2 = 512
    model2 = PPO2(CnnPolicy, env2, verbose=1, n_steps=batch_size2, # seed=314, n_cpu_tf_sess=1,
             gamma=0.99, learning_rate=0.00025, nminibatches=epochs, ent_coef=0.01, vf_coef=0.5) 
    model2.load_parameters(param_list, exact_match=True)
    model2.learn(total_timesteps = updates*batch_size2, log_interval=1, callback=eval_callback)
else:
    model.set_env(env2)
    model.learn(total_timesteps = updates*batch_size, log_interval=1, callback=eval_callback)


Track generation: 1276..1598 -> 322-tiles track, complex 12
--------------------------------------
| approxkl           | 0.0033028494  |
| clipfrac           | 0.032226562   |
| explained_variance | -0.265        |
| fps                | 49            |
| n_updates          | 1             |
| policy_entropy     | 0.19470961    |
| policy_loss        | -0.0033594884 |
| serial_timesteps   | 256           |
| time_elapsed       | 0.004         |
| total_timesteps    | 256           |
| value_loss         | 1.9427452     |
--------------------------------------
1  cut by time without progress. Steps 434  %advance 56.0  played reward 12.15  last penalty -20
--------------------------------------
| approxkl           | 0.005588497   |
| clipfrac           | 0.080078125   |
| explained_variance | 0.692         |
| fps                | 48            |
| n_updates          | 2             |
| policy_entropy     | 0.42564395    |
| policy_loss        | -0.0078270985 |
| serial_timesteps   | 5

-------------------------------------
| approxkl           | 0.0028798399 |
| clipfrac           | 0.041992188  |
| explained_variance | 0.479        |
| fps                | 47           |
| n_updates          | 15           |
| policy_entropy     | 0.32011896   |
| policy_loss        | -0.01108386  |
| serial_timesteps   | 3840         |
| time_elapsed       | 71.5         |
| total_timesteps    | 3840         |
| value_loss         | 0.33724985   |
-------------------------------------
-------------------------------------
| approxkl           | 0.0032357692 |
| clipfrac           | 0.037109375  |
| explained_variance | 0.635        |
| fps                | 50           |
| n_updates          | 16           |
| policy_entropy     | 0.52155906   |
| policy_loss        | -0.009424256 |
| serial_timesteps   | 4096         |
| time_elapsed       | 76.9         |
| total_timesteps    | 4096         |
| value_loss         | 3.49379      |
-------------------------------------
12  cut by t

21  cut by time without progress. Steps 270  %advance 26.4  played reward -1.04  last penalty -20
-------------------------------------
| approxkl           | 0.008311932  |
| clipfrac           | 0.08984375   |
| explained_variance | 0.927        |
| fps                | 51           |
| n_updates          | 30           |
| policy_entropy     | 0.7663045    |
| policy_loss        | -0.012777058 |
| serial_timesteps   | 7680         |
| time_elapsed       | 147          |
| total_timesteps    | 7680         |
| value_loss         | 2.6094995    |
-------------------------------------
22  cut by time without progress. Steps 395  %advance 50.4  played reward 10.44  last penalty -20
-------------------------------------
| approxkl           | 0.011686539  |
| clipfrac           | 0.10449219   |
| explained_variance | 0.947        |
| fps                | 52           |
| n_updates          | 31           |
| policy_entropy     | 0.77260906   |
| policy_loss        | -0.023067785 |
| seri

33  cut by time without progress. Steps 282  %advance 28.9  played reward 0.24  last penalty -20
-------------------------------------
| approxkl           | 0.0062122466 |
| clipfrac           | 0.09667969   |
| explained_variance | 0.848        |
| fps                | 52           |
| n_updates          | 45           |
| policy_entropy     | 0.86296695   |
| policy_loss        | -0.011479665 |
| serial_timesteps   | 11520        |
| time_elapsed       | 222          |
| total_timesteps    | 11520        |
| value_loss         | 5.547432     |
-------------------------------------
34  cut by time without progress. Steps 355  %advance 33.0  played reward -3.0  last penalty -20
-------------------------------------
| approxkl           | 0.006688629  |
| clipfrac           | 0.083984375  |
| explained_variance | 0.708        |
| fps                | 51           |
| n_updates          | 46           |
| policy_entropy     | 0.82894206   |
| policy_loss        | -0.011867457 |
| serial

-------------------------------------
| approxkl           | 0.013080011  |
| clipfrac           | 0.10839844   |
| explained_variance | 0.927        |
| fps                | 51           |
| n_updates          | 56           |
| policy_entropy     | 0.44385624   |
| policy_loss        | -0.011398336 |
| serial_timesteps   | 14336        |
| time_elapsed       | 370          |
| total_timesteps    | 14336        |
| value_loss         | 0.3031834    |
-------------------------------------
--------------------------------------
| approxkl           | 0.0015534937  |
| clipfrac           | 0.009765625   |
| explained_variance | 0.88          |
| fps                | 52            |
| n_updates          | 57            |
| policy_entropy     | 0.74712306    |
| policy_loss        | -0.0010020328 |
| serial_timesteps   | 14592         |
| time_elapsed       | 375           |
| total_timesteps    | 14592         |
| value_loss         | 1.7354673     |
--------------------------------------

...

8  cut by time without progress. Steps 218  %advance 25.0  played reward 2.54  last penalty -20
-------------------------------------
| approxkl           | 0.00403187   |
| clipfrac           | 0.048828125  |
| explained_variance | 0.827        |
| fps                | 49           |
| n_updates          | 441          |
| policy_entropy     | 0.6507344    |
| policy_loss        | -0.010536233 |
| serial_timesteps   | 112896       |
| time_elapsed       | 3.38e+03     |
| total_timesteps    | 112896       |
| value_loss         | 9.039753     |
-------------------------------------
-------------------------------------
| approxkl           | 0.006516384  |
| clipfrac           | 0.07910156   |
| explained_variance | 0.85         |
| fps                | 47           |
| n_updates          | 442          |
| policy_entropy     | 0.70769954   |
| policy_loss        | -0.011193118 |
| serial_timesteps   | 113152       |
| time_elapsed       | 3.38e+03     |
| total_timesteps    | 113152 

15  Finalized in Steps 526  with return=total reward 146.64242424242414
-------------------------------------
| approxkl           | 0.009318103  |
| clipfrac           | 0.15234375   |
| explained_variance | -0.0211      |
| fps                | 54           |
| n_updates          | 453          |
| policy_entropy     | 1.0344791    |
| policy_loss        | 0.0047072684 |
| serial_timesteps   | 115968       |
| time_elapsed       | 3.57e+03     |
| total_timesteps    | 115968       |
| value_loss         | 149.25044    |
-------------------------------------
-------------------------------------
| approxkl           | 0.017238196  |
| clipfrac           | 0.22070312   |
| explained_variance | 0.539        |
| fps                | 44           |
| n_updates          | 454          |
| policy_entropy     | 1.0219117    |
| policy_loss        | -0.016089197 |
| serial_timesteps   | 116224       |
| time_elapsed       | 3.57e+03     |
| total_timesteps    | 116224       |
| value_loss    

--------------------------------------
| approxkl           | 0.0029810343  |
| clipfrac           | 0.038085938   |
| explained_variance | 0.725         |
| fps                | 50            |
| n_updates          | 469           |
| policy_entropy     | 0.6612879     |
| policy_loss        | -0.0072628623 |
| serial_timesteps   | 120064        |
| time_elapsed       | 3.65e+03      |
| total_timesteps    | 120064        |
| value_loss         | 0.09410011    |
--------------------------------------
22  Finalized in Steps 609  with return=total reward 138.34242424242422
-------------------------------------
| approxkl           | 0.009230081  |
| clipfrac           | 0.12109375   |
| explained_variance | 0.0974       |
| fps                | 54           |
| n_updates          | 470          |
| policy_entropy     | 0.6785421    |
| policy_loss        | -0.008341743 |
| serial_timesteps   | 120320       |
| time_elapsed       | 3.65e+03     |
| total_timesteps    | 120320       |
| v

32  Finalized in Steps 498  with return=total reward 149.44242424242418
--------------------------------------
| approxkl           | 0.0032124014  |
| clipfrac           | 0.036132812   |
| explained_variance | 0.00559       |
| fps                | 52            |
| n_updates          | 484           |
| policy_entropy     | 0.5737371     |
| policy_loss        | -0.0054955482 |
| serial_timesteps   | 123904        |
| time_elapsed       | 3.72e+03      |
| total_timesteps    | 123904        |
| value_loss         | 141.28648     |
--------------------------------------
33  cut by time without progress. Steps 248  %advance 21.9  played reward -3.86  last penalty -20
-------------------------------------
| approxkl           | 0.013255206  |
| clipfrac           | 0.140625     |
| explained_variance | 0.483        |
| fps                | 49           |
| n_updates          | 485          |
| policy_entropy     | 0.81978047   |
| policy_loss        | 0.0006466417 |
| serial_timesteps 

-------------------------------------
| approxkl           | 0.009337889  |
| clipfrac           | 0.1484375    |
| explained_variance | 0.0372       |
| fps                | 50           |
| n_updates          | 499          |
| policy_entropy     | 0.91562873   |
| policy_loss        | -0.010252877 |
| serial_timesteps   | 127744       |
| time_elapsed       | 3.8e+03      |
| total_timesteps    | 127744       |
| value_loss         | 141.6658     |
-------------------------------------
41  cut by time without progress. Steps 310  %advance 39.3  played reward 7.73  last penalty -20
1  cut by time without progress. Steps 323  %advance 51.3  played reward 17.99  last penalty -20
2  cut by time without progress. Steps 323  %advance 51.3  played reward 17.99  last penalty -20
3  cut by time without progress. Steps 299  %advance 38.0  played reward 7.45  last penalty -20
4  cut by time without progress. Steps 556  %advance 98.8  played reward 42.14  last penalty -20
5  cut by time without

In [15]:
#import pickle
if new_mod:
    file = root+'c{:d}_f{:d}_s{:d}_{}_u{:d}_e{:d}_p{}_bs{:d}'.format(game_color,fpst,skip,indicators,use,ept,patience,batch_size)
    model2.save(file, cloudpickle=True)
    param_list=model2.get_parameter_list()
else:
    model.save(file+'II', cloudpickle=True)
    param_list=model.get_parameter_list()


In [16]:
env2.close()
env_test.close()

In [25]:
## Enjoy last trained policy

if agent=='CarRacing-v2':  #create an independent test environment
    env3 = gym.make(agent, seed=None, 
        game_color=game_color,
        use_track = 2,       
        episodes_per_track = 1,  
        tr_complexity = 12, 
        patience = 5.0,
        discre = actions,
        indicators = True,
        frames_per_state = fpst,
        skip_frames = skip   )
else:
    env3 = gym.make(agent)

env3 = DummyVecEnv([lambda: env3])
obs = env3.reset()
print(obs.shape)        

done = False
pasos = 0
_states=None

while not done and pasos<1002:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env3.step(action)
    env3.render()
    pasos+=1
    
env3.close()
print()
print(reward, done, info, pasos)

Track generation: 984..1233 -> 249-tiles track, complex 12
(1, 96, 96, 4)
1  cut by time without progress. Steps 501  %advance 70.9  played reward 20.16  last penalty -20

[-20.] [ True] [{'terminal_observation': array([[[161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        ...,
        [  0.   ,   0.   , 176.549, 161.874],
        [  0.   ,   0.   ,   0.   , 161.874],
        [  0.   ,   0.   ,   0.   , 161.874]],

       [[161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        ...,
        [  0.   ,   0.   , 176.549, 161.874],
        [  0.   ,   0.   ,   0.   , 161.874],
        [  0.   ,   0.   ,   0.   , 161.874]],

       [[161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        [161.874, 161.874, 161.874, 161.874],
        ...,
        [  0.   ,   0.   , 176.549, 161.874],
        

In [26]:
## Enjoy best eval_policy

obs = env3.reset()
print(obs.shape)        

## Load bestmodel from eval
#if not isinstance(model_test, PPO2):
model_test = PPO2.load(eval_log+'best_model', env3)

done = False
pasos = 0
_states=None

while not done and pasos<1002:
    action, _states = model_test.predict(obs, deterministic=True)
    obs, reward, done, info = env3.step(action)
    env3.render()
    pasos+=1
    
env3.close()
print()
print(reward, done, pasos)
print(action, _states)

Track generation: 1088..1363 -> 275-tiles track, complex 12
(1, 96, 96, 4)
0  Finalized in Steps 831  with return=total reward 116.1700729927009

[100.26496] [ True] 831
[3] None


In [19]:
model_test.save(file+'_evalbest', cloudpickle=True)

In [20]:
env2.close()

In [21]:
env3.close()

In [22]:
env_test.close()

In [23]:
print(action, _states)

[0] None


In [24]:
obs.shape

(1, 96, 96, 4)